In [36]:
from autocat.surface import generate_surface_structures

from autocat.adsorption import generate_rxn_structures
from autocat.adsorption import generate_molecule_object

from autocat.data.intermediates import ORR_INTERMEDIATE_NAMES

In this example we show how to use `AutoCat` to generate adsorption structures given a surface structure

# Generating Reaction Structures

Let's start by making a `Pt111` slab for demonstration purposes. But in general this can be any surface you'd like as long as you have it in the form of an `ase.Atoms` object or written to disk in an `ase` readable format.

In [5]:
slab_dictionary = generate_surface_structures(
    species_list = ["Pt"],
    facets = {"Pt": ["111"]},
    n_fixed_layers = 2
)

clean_slab = slab_dictionary["Pt"]["fcc111"]["structure"]

Now that we have our clean slab, we can start adsorbing molecules onto the surface.

To fully characterize this surface for its activity toward evolving hydrogen (HER), we'd need to adsorb `H` onto every symmetry site of the surface. As the choice of `Pt111` was arbitrary, the function demo'd here works for any surface (through `pymatgen`'s implementation of Delaunay Triangulation).

In [6]:
h_adsorption_structure_dictionary = generate_rxn_structures(
    clean_slab,
    all_sym_sites = True, # to consider all identified sites
    ads = ["H"],
    height = {"H" : 1.5}, # manually specify height. default guess based on covalent radii of nearest neighbors
    refs = ["H2"], # reference molecule for the reaction
    write_to_disk = False
)

This will generate a dictionary of all generated adsorption structures with the following structure:

- Adsorbate Names
    - Symmetry Site types (ie. hollow, ontop, bridge) or Custom Label
        - `x-y` coordinate of each site
            - `ase.Atoms` structure
            - Path to structure file (in the `ase.traj` format)
- references
    - Reference Molecule Name
        - `ase.Atoms` structure
        - Path to structure file (in the `ase.traj` format)

Here we have all three types of symmetry sites present

In [27]:
print(h_adsorption_structure_dictionary["H"].keys())

dict_keys(['ontop', 'bridge', 'hollow'])


And we can confirm that it identified both hollow sites:

In [32]:
print(h_adsorption_structure_dictionary["H"]["hollow"].keys())

dict_keys(['6.93_5.601', '9.702_4.001'])


Instead of exhaustively considering all sites, it can be restricted to specific types via `site_type`. 

Or alternatively, if we want to consider only manually specified sites, that can be done via `sites`. When specifying the sites manually in this way, we need to provide them as a dictionary with keys as to how we'd like the site labelled. This is solely used for organizing the output dictionary

In [43]:
h_manual_adsorption_structure_dictionary = generate_rxn_structures(
    clean_slab,
    all_sym_sites = False,
    ads = ["H"],
    sites = {"custom": [(0.,0.)]},
    write_to_disk = False
)

In [45]:
print(h_manual_adsorption_structure_dictionary["H"].keys())
print(h_manual_adsorption_structure_dictionary["H"]["custom"].keys())

dict_keys(['custom'])
dict_keys(['0.0_0.0'])


`AutoCat` also has some defaults for generating structures for considering the Oxygen Reduction/Evolution Reaction as well as Nitrogen Reduction. These can be found within `autocat.data.intermediates`. Let's generate the ORR adsorption structures on this slab as an example.

In [34]:
orr_adsorption_structure_dictionary = generate_rxn_structures(
    clean_slab,
    all_sym_sites = True, # to consider all identified sites (can also manually specify via `sites`)
    ads = ORR_INTERMEDIATE_NAMES,
    refs = ["H2", "H2O"], # reference molecules for the reaction
    write_to_disk = False
)

This places all of the relevant adsorbate molecules at all of the identified sites.

In [35]:
print(orr_adsorption_structure_dictionary.keys())

dict_keys(['OOH', 'O', 'OH', 'references'])


It's important to note that if you already have the adsorbate molecule you'd like to consider as an `ase.Atoms` object, that can be supplied as well. We are going to use `autocat.adsorption.generate_molecule_object` to generate an example, but this can be anything (e.g. an `*.sdf` read by `ase.io.read`)

In [39]:
nh2_mol = generate_molecule_object("NH2").get("structure")

nh2_adsorption_structure_dictionary = generate_rxn_structures(
    clean_slab,
    all_sym_sites = True,
    ads = [nh2_mol],
)

In [41]:
print(nh2_adsorption_structure_dictionary.keys())

dict_keys(['H2N'])
